In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder  # for scenery dataset
from torch.utils.tensorboard import SummaryWriter

# DCGAN-compatible Discriminator
class Discriminator(nn.Module):
    def __init__(self, img_channels, features_d=64):
        super().__init__()
        self.disc = nn.Sequential(
            nn.Conv2d(img_channels, features_d, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(features_d, features_d * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_d * 2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(features_d * 2, features_d * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_d * 4),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(features_d * 4, features_d * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_d * 8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(features_d * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.disc(x)


# DCGAN-compatible Generator
class Generator(nn.Module):
    def __init__(self, z_dim, img_channels, features_g=64):
        super().__init__()
        self.gen = nn.Sequential(
            nn.ConvTranspose2d(z_dim, features_g * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(features_g * 8),
            nn.ReLU(True),

            nn.ConvTranspose2d(features_g * 8, features_g * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_g * 4),
            nn.ReLU(True),

            nn.ConvTranspose2d(features_g * 4, features_g * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_g * 2),
            nn.ReLU(True),

            nn.ConvTranspose2d(features_g * 2, features_g, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_g),
            nn.ReLU(True),

            nn.ConvTranspose2d(features_g, img_channels, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.gen(x)


In [ ]:
from google.colab import files
files.upload()  # Then upload your kaggle.json

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!pip install kaggle
!kaggle datasets download -d arnaud58/landscape-pictures
!unzip landscape-pictures.zip -d dataset/

In [ ]:
!mkdir dataset/scenery
!mv dataset/*.jpg dataset/scenery/


In [ ]:

device = "cuda" if torch.cuda.is_available() else "cpu"
lr = 3e-4
z_dim = 128
image_channels = 3
image_size = 64
batch_size = 128
num_epochs = 100

# Init models
gen = Generator(z_dim=z_dim, img_channels=3, features_g=64).to(device)
disc = Discriminator(img_channels=3, features_d=64).to(device)


# Fixed noise to visualize progress
fixed_noise = torch.randn(batch_size, z_dim, 1, 1).to(device)

# Data transform for images
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5 for _ in range(image_channels)],
                         [0.5 for _ in range(image_channels)]),
])

# If images are inside a folder like "dataset/landscape"
dataset = ImageFolder(root="dataset", transform=transform)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Optimizers and loss
opt_disc = optim.Adam(disc.parameters(), lr=lr, betas=(0.5, 0.999))
opt_gen = optim.Adam(gen.parameters(), lr=lr, betas=(0.5, 0.999))
criterion = nn.BCELoss()


In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs


In [ ]:
writer_fake = SummaryWriter(f"runs/DCGAN_Dataset/fake")
writer_real = SummaryWriter(f"runs/DCGAN_Dataset/real")
step = 0
real_label = 0.9
fake_label = 0.0
for epoch in range(num_epochs):
    for batch_idx, (real, _) in enumerate(loader):
        real = real.to(device)
        batch_size = real.size(0)

        # Train Discriminator
        noise = torch.randn(batch_size, z_dim, 1, 1).to(device)
        fake = gen(noise)

        disc_real = disc(real).view(-1)
        lossD_real = criterion(disc_real, torch.full_like(disc_real, real_label))



        disc_fake = disc(fake.detach()).view(-1)  # detach so gen doesn't get gradients
        lossD_fake = criterion(disc_fake, torch.full_like(disc_fake, fake_label))

        lossD = (lossD_real + lossD_fake) / 2
        disc.zero_grad()
        lossD.backward()
        opt_disc.step()

        # Train Generator
        output = disc(fake).view(-1)
        lossG = criterion(output, torch.ones_like(output))
        gen.zero_grad()
        lossG.backward()
        opt_gen.step()

        # Logging
        if batch_idx == 0:
            print(
                f"Epoch [{epoch}/{num_epochs}] Batch {batch_idx}/{len(loader)} "
                f"Loss D: {lossD:.4f}, loss G: {lossG:.4f}"
            )

            with torch.no_grad():
                fake = gen(fixed_noise)
                img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
                img_grid_real = torchvision.utils.make_grid(real, normalize=True)

                writer_fake.add_image("Fake Images", img_grid_fake, global_step=step)
                writer_real.add_image("Real Images", img_grid_real, global_step=step)

                # ✅ Loss curve logging
                writer_fake.add_scalar("Loss/Generator", lossG.item(), global_step=step)
                writer_fake.add_scalar("Loss/Discriminator", lossD.item(), global_step=step)

                step += 1
        # Save model every 5 epochs
    if epoch % 5 == 0:
        torch.save(gen.state_dict(), f"generator_epoch{epoch}.pth")


